In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
import re
from nltk import WordNetLemmatizer
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras import metrics
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from sklearn.model_selection import train_test_split


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Kiril\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Kiril\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Kiril\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
train,test=pd.read_csv(r'C:\Users\Kiril\projects\nlp_tweets\tweet-sentiment-extraction\train.csv'),pd.read_csv(r'C:\Users\Kiril\projects\nlp_tweets\tweet-sentiment-extraction\test.csv')

In [3]:
test.head()

,textID,text,sentiment
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral
1,96d74cb729,Shanghai is also really exciting (precisely -...,positive
2,eee518ae67,"Recession hit Veronique Branquinho, she has to...",negative
3,01082688c6,happy bday!,positive
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!,positive


In [4]:
train.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [5]:
data=pd.concat([train,test])

In [6]:
def preprocess_text(data:pd.DataFrame):
    data["text"]=data["text"].str.lower()
    
    stop_words=stopwords.words("english")
    
    data["text"]=data["text"].astype(str)
    data["text"]=data["text"].apply(lambda x: ' '.join(word for word in x.split() if word not in stop_words))
    
    data["text"]=data["text"].apply(lambda x: re.sub(r"http\S+","",x))
    data["text"]=data["text"].apply(lambda x: re.sub(r"www\.\S+","",x))
    
    data["text"] = data["text"].apply(lambda x: re.sub(r"\d", "", x))
    data["text"] = data["text"].apply(lambda x: re.sub(r"\w+@\w+\.com", "", x))

    data["text"] = data["text"].apply(lambda x: re.sub(r"[.,;:!\?\"'`]", "", x))
    data["text"] = data["text"].apply(lambda x: re.sub(r"[@#\$%^&*\(\)\\/\+-_=\[\]\{\}<>]", "", x))
    
    lemmantizer=WordNetLemmatizer()
    
    data["text"] = data["text"].apply(lambda x: " ".join(lemmantizer.lemmatize(word,"v") for word in x.split( )))
    
    tokenizer=Tokenizer(num_words=5000,split=" ")
    tokenizer.fit_on_texts(data["text"].values)
    
    X=tokenizer.texts_to_sequences(data["text"].values)
    X=pad_sequences(X)
    
    Y=pd.get_dummies(data["sentiment"].values)
    return X,Y    

In [7]:
samples,labels=preprocess_text(data)

In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(samples, labels, test_size=0.2, random_state=42)

In [9]:
model=Sequential()
model.add(Embedding(5000,128,input_length=X_train.shape[1]))
model.add(LSTM(128,dropout=0.2))
model.add(Dense(3,activation="softmax"))
model.compile(loss='categorical_crossentropy',optimizer="adam",metrics=["accuracy"])

In [10]:
model.fit(X_train, Y_train, epochs=10, batch_size=32)

Epoch 1/10
776/776 [==============================] - 19s 21ms/step - loss: 0.7886 - accuracy: 0.6507
Epoch 2/10
776/776 [==============================] - 15s 19ms/step - loss: 0.6174 - accuracy: 0.7497
Epoch 3/10
776/776 [==============================] - 14s 18ms/step - loss: 0.5479 - accuracy: 0.7803
Epoch 4/10
776/776 [==============================] - 14s 18ms/step - loss: 0.4844 - accuracy: 0.8111
Epoch 5/10
776/776 [==============================] - 14s 18ms/step - loss: 0.4196 - accuracy: 0.8363
Epoch 6/10
776/776 [==============================] - 14s 18ms/step - loss: 0.3640 - accuracy: 0.8590
Epoch 7/10
776/776 [==============================] - 14s 18ms/step - loss: 0.3137 - accuracy: 0.8811
Epoch 8/10
776/776 [==============================] - 14s 18ms/step - loss: 0.2712 - accuracy: 0.8972
Epoch 9/10
776/776 [==============================] - 14s 18ms/step - loss: 0.2334 - accuracy: 0.9110
Epoch 10/10
776/776 [==============================] - 15s 20ms/step - loss: 0.203

In [14]:

score, acc = model.evaluate(X_test, Y_test, verbose=1)
print("Accuracy: %.2f" % (acc*100))

194/194 [==============================] - 1s 6ms/step - loss: 1.4297 - accuracy: 0.6687
Accuracy: 66.87
